In [9]:
import pandas as pd
import numpy as np

# making pandas dataframe of `Current Status` logs
# and getting number of rows
logs = pd.read_csv('./current_status.csv')
logs_rows = logs.index
logs_num_rows = len(logs_rows)

# making pandas dataframe of `Past Status` logs
# which is indexed by IP
past_logs = pd.read_csv('./past_status.csv')
past_by_IP = past_logs.set_index('IP Address')
past_IPs = list(past_by_IP.index)

In [10]:
# helper functions for reformatting Plugin Outputs

import copy

# takes original 'Plugin Output' as string
# groups lines into arrays of ['Path : ...', 'Installed...', 'Fixed...']
# returns array of those arrays  
def group_paths(po):
    po = po.split("\n")[1:]
    answer = []
    current_group = []
    for p in po:
        if (p != ''):
            current_group.append(p)
        if p[2:3] == "F":
            answer.append(copy.copy(current_group))
            current_group.clear()
    return answer

# takes original 'Plugin Output' as string
# returns dictionary of (string, string): string
# where keys are pairs lines for installed & fixed versions
# and values are the arrays of paths with those versions
def combined_groups(po):
    grouped_po = group_paths(po)
    answer = {}
    for group in grouped_po:
        path = group[0]
        inst_v = group[1]
        fixed_v = group[2]
        versions = (inst_v, fixed_v)
        if versions in answer.keys():
            answer[versions].append(path)
        else:
            answer[versions] = [path]
    return answer

# takes original 'Plugin Output' as string
# returns new shortened format of 'Plugin Output' string
def reformatted_po(po):
    answer = "Plugin Output:\n"
    groups = combined_groups(po)
    for key in groups.keys():
        (inst_v, fixed_v) = key
        for path in groups[key]:
            answer = f'{answer}{path}\n'
        answer = f'{answer}{inst_v}\n{fixed_v}\n\n'
    return answer

In [11]:
# filling in columns of `Current Status` logs
# and reformatting its Plugin Output column
unknown_ips = []

# arrays to store new columns
apps = [""]*logs_num_rows
contacts = [""]*logs_num_rows
new_pos = [""]*logs_num_rows
vulnerable = [""]*logs_num_rows
response = [""]*logs_num_rows

for i in logs_rows:
    ip = logs.loc[i, 'IP Address']
    po = logs.loc[i, 'Plugin Output']
    
    # filling in Application, Contact, Vulnerable?, and RESPONSE columns
    if ip in past_IPs: 
        apps[i] = past_by_IP.loc[ip]['Application']
        contacts[i] = past_by_IP.loc[ip]['Contact']
        vulnerable[i] = past_by_IP.loc[ip]['Vulnerable?']
        response[i] = past_by_IP.loc[ip]['RESPONSE']
    else:
        unknown_ips.append(ip)
        
    # reformatting Plugin Output column
    new_po = reformatted_po(po)
    new_pos[i] = new_po
    
print("Unknown IPs")
print(unknown_ips)

# making csv file with new Application, Contact, & Plugin Output columns
logs.drop('Plugin Output', 1, inplace=True)
logs.insert(2, 'Application', apps)
logs.insert(3, 'Contact', contacts)
logs.insert(4, 'Plugin Output', new_pos)
logs.to_csv('./new_current_status.csv')

# making csv file with added Vulnerable? & RESPONSE columns
logs.insert(5, 'Vulnerable?', vulnerable)
logs.insert(6, 'RESPONSE', response)
logs.to_csv('./new_current_status_extra.csv')

['136.167.14.246']


In [8]:
import copy

def short_path(path):
    return path[:36] + "..."
    
def print_groups(array, name):
    print(f'{name}:')
    for elem in array:
        path = elem[0]
        i = elem[1]
        f = elem[2]
        print(f'[{short_path(path)}\n{i}\n{f}]')
        
def print_po(combined_groups, name):
    print(f'{name}:')
    for key in combined_groups.keys():
#         print(key)
#         print(combined_groups[key])
        (inst_v, fixed_v) = key
        for path in combined_groups[key]:
            print(short_path(path))
        print(f'{inst_v}\n{fixed_v}')

def group_paths(po):
    po = po.split("\n")[1:]
    answer = []
    current_arr = []
    for p in po:
        if (p != ''):
            current_arr.append(p)
        if p[2:3] == "F":
            answer.append(copy.copy(current_arr))
            current_arr.clear()
    return answer

def combined_groups(po):
    grouped_po = group_paths(po)
    answer = {}
    for group in grouped_po:
        path = group[0]
        inst_v = group[1]
        fixed_v = group[2]
        #print(f'current group:\n{path}\n{inst_v} {fixed_v}')
        versions = (inst_v, fixed_v)
        if versions in answer.keys():
            answer[versions].append(path)
        else:
            answer[versions] = [path]
    return answer

def reformatted_po(po):
    answer = "Plugin Output:\n"
    groups = combined_groups(po)
    for key in groups.keys():
        (inst_v, fixed_v) = key
        for path in groups[key]:
            answer = f'{answer}{path}\n'
        answer = f'{answer}{inst_v}\n{fixed_v}\n\n'
    return answer

for i in range(0,5):
    print(i)
    po = logs.loc[i, 'Plugin Output']
    print(po)

    grouped_po = group_paths(po)
#     print_groups(grouped_po, 'paths')

    combined_po = combined_groups(po)
#     print(combined_po)
#     print_po(combined_po, 'combined paths')

    print("")
    new_po = reformatted_po(po)
    print(new_po)
    print("\n\n\n")

0
Plugin Output:
  Path              : /usr/nsh/BSA21.3.00.38RU2Backup_06.01.2022_13.56.52/log4j-core-2.13.1.jar
  Installed version : 2.13.1
  Fixed version     : 2.17.1



Plugin Output:
  Path              : /usr/nsh/BSA21.3.00.38RU2Backup_06.01.2022_13.56.52/log4j-core-2.13.1.jar
  Installed version : 2.13.1
  Fixed version     : 2.17.1






1
Plugin Output:
  Path              : /home/delphixutl/dlpx-masking-initializer-1.1.0/lib/log4j-core-2.10.0.jar
  Installed version : 2.10.0
  Fixed version     : 2.12.4



Plugin Output:
  Path              : /home/delphixutl/dlpx-masking-initializer-1.1.0/lib/log4j-core-2.10.0.jar
  Installed version : 2.10.0
  Fixed version     : 2.12.4






2
Plugin Output:
  Path              : /usr/nsh/NSH/br/stdlib/log4j-core-2.13.1.jar
  Installed version : 2.13.1
  Fixed version     : 2.17.1



Plugin Output:
  Path              : /usr/nsh/NSH/br/stdlib/log4j-core-2.13.1.jar
  Installed version : 2.13.1
  Fixed version     : 2.17.1






3
Plugin Ou